In [46]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot


from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

pd.options.display.float_format = "{:,.4f}".format

In [47]:
FILENAME = "mnist.pkl.gz"
DATA_PATH = Path("/Users/arman")
PATH = DATA_PATH / "Downloads"
with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), (x_test, y_test)) = pickle.load(f, encoding="latin-1")

In [48]:
#Classification Model

In [49]:
class Net2nn(nn.Module):
    def __init__(self):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(784,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,10)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [50]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [51]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [52]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [53]:
# This function compares the accuracy of the main model 
# and the local model running on each node.
def compare_local_and_merged_model_performance(number_of_samples):
    accuracy_table=pd.DataFrame(data=np.zeros((number_of_samples,3)), columns=["sample", "local_ind_model", "merged_main_model"])
    for i in range (number_of_samples):
    
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        individual_loss, individual_accuracy = validation(model, test_dl, criterion)
        main_loss, main_accuracy =validation(main_model, test_dl, main_criterion )
    
        accuracy_table.loc[i, "sample"]="sample "+str(i)
        accuracy_table.loc[i, "local_ind_model"] = individual_accuracy
        accuracy_table.loc[i, "merged_main_model"] = main_accuracy

    return accuracy_table

In [54]:
#Functions for Federated Averaging 

In [55]:
#Optimizers are algorithms or methods used to minimize an
# error function(loss function)or to maximize the efficiency of production. 

# This function creates a model, optimizer and loss function for each node.
def create_model_optimizer_criterion_dict(number_of_samples):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_samples):
        model_name="model"+str(i)
        model_info=Net2nn()
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
        
    return model_dict, optimizer_dict, criterion_dict

In [56]:
# This function takes the average of the weights in individual nodes.

def get_averaged_weights(model_dict, number_of_samples):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
    
    with torch.no_grad():
    
    
        for i in range(number_of_samples):
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()

        
        fc1_mean_weight =fc1_mean_weight/number_of_samples
        fc1_mean_bias = fc1_mean_bias/ number_of_samples
    
        fc2_mean_weight =fc2_mean_weight/number_of_samples
        fc2_mean_bias = fc2_mean_bias/ number_of_samples
    
        fc3_mean_weight =fc3_mean_weight/number_of_samples
        fc3_mean_bias = fc3_mean_bias/ number_of_samples
    
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias

In [57]:
# This function sends the averaged weights of individual nodes 
# to the main model and sets them as the new weights of the main model. ( calls def get_averaged_weights(model_dict, number_of_samples))

def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_samples):
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias = get_averaged_weights(model_dict, number_of_samples=number_of_samples)
    with torch.no_grad():
        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
    return main_model

In [58]:
# This function sends the parameters of the main model to the nodes.
# The following data should send from SERVER to each node
# So we should save them on a file

def send_main_model_to_nodes_and_update_model_dict(main_model,want_print):
    with torch.no_grad():
            torch.save(main_model.fc1.weight.data.clone(), '/Users/arman/Downloads/Security-project/Main_Model_Parameters/main_model_fc1_weight_data.pt')
            torch.save(main_model.fc2.weight.data.clone(), '/Users/arman/Downloads/Security-project/Main_Model_Parameters/main_model_fc2_weight_data.pt')
            torch.save(main_model.fc3.weight.data.clone(), '/Users/arman/Downloads/Security-project/Main_Model_Parameters/main_model_fc3_weight_data.pt')
            
            torch.save(main_model.fc1.bias.data.clone(), '/Users/arman/Downloads/Security-project/Main_Model_Parameters/main_model_fc1_bias_data.pt')
            torch.save(main_model.fc2.bias.data.clone(), '/Users/arman/Downloads/Security-project/Main_Model_Parameters/main_model_fc2_bias_data.pt')
            torch.save(main_model.fc3.bias.data.clone(), '/Users/arman/Downloads/Security-project/Main_Model_Parameters/main_model_fc3_bias_data.pt')

            if (want_print == 1):
                print(main_model.fc1.weight.data.clone())
                print("--------------------------------------------------------")
                print(main_model.fc2.weight.data.clone())
                print("--------------------------------------------------------")
                print(main_model.fc3.weight.data.clone())
                print("--------------------------------------------------------")

                print(main_model.fc1.bias.data.clone())
                print("--------------------------------------------------------")
                print(main_model.fc2.bias.data.clone())
                print("--------------------------------------------------------")
                print(main_model.fc3.bias.data.clone()) 



In [59]:
# This function downloads all of the local models

def download_local_model_from_each_node(number_of_samples):
    with torch.no_grad():
        for i in range(number_of_samples):
            model_dict[name_of_models[i]].fc1.weight.data = torch.load('/Users/arman/Downloads/Security-project/Local_Model_Parameters/local_model_'+str(i)+'_fc1_weight_data.pt')
            model_dict[name_of_models[i]].fc2.weight.data = torch.load('/Users/arman/Downloads/Security-project/Local_Model_Parameters/local_model_'+str(i)+'_fc2_weight_data.pt')
            model_dict[name_of_models[i]].fc3.weight.data = torch.load('/Users/arman/Downloads/Security-project/Local_Model_Parameters/local_model_'+str(i)+'_fc3_weight_data.pt')
            
            model_dict[name_of_models[i]].fc1.bias.data = torch.load('/Users/arman/Downloads/Security-project/Local_Model_Parameters/local_model_'+str(i)+'_fc1_bias_data.pt')
            model_dict[name_of_models[i]].fc2.bias.data = torch.load('/Users/arman/Downloads/Security-project/Local_Model_Parameters/local_model_'+str(i)+'_fc2_bias_data.pt')
            model_dict[name_of_models[i]].fc3.bias.data = torch.load('/Users/arman/Downloads/Security-project/Local_Model_Parameters/local_model_'+str(i)+'_fc3_bias_data.pt')
    
    return model_dict

In [60]:
x_train, y_train, x_valid, y_valid,x_test, y_test = map(torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test))
number_of_samples = 3
learning_rate = 0.01
numEpoch = 10
batch_size = 32
momentum = 0.9

train_amount = 4500
valid_amount = 900
test_amount = 900
print_amount = 3

In [61]:
# Main model is created


In [62]:
main_model = Net2nn()
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

In [63]:
print(main_model.fc2.weight[0:1,0:5])



tensor([[ 0.0630,  0.0016, -0.0621,  0.0142,  0.0037]],
       grad_fn=<SliceBackward0>)


In [64]:
# The following data should send from SERVER to each node
# So we should save them on a file

send_main_model_to_nodes_and_update_model_dict(main_model,1) # if 1, then print weight and bias


tensor([[-0.0333,  0.0047,  0.0323,  ...,  0.0350, -0.0311,  0.0340],
        [-0.0056, -0.0225,  0.0161,  ..., -0.0231, -0.0351,  0.0302],
        [-0.0238,  0.0057,  0.0228,  ...,  0.0274, -0.0056,  0.0074],
        ...,
        [-0.0134, -0.0037, -0.0340,  ..., -0.0118, -0.0305, -0.0296],
        [ 0.0282,  0.0043, -0.0067,  ...,  0.0259, -0.0157,  0.0014],
        [ 0.0088, -0.0201,  0.0050,  ...,  0.0269, -0.0067,  0.0154]])
--------------------------------------------------------
tensor([[ 0.0630,  0.0016, -0.0621,  ...,  0.0556,  0.0075,  0.0686],
        [ 0.0342,  0.0128,  0.0339,  ...,  0.0117, -0.0200, -0.0275],
        [-0.0056,  0.0551, -0.0672,  ..., -0.0593,  0.0421,  0.0384],
        ...,
        [ 0.0096,  0.0040,  0.0166,  ...,  0.0471,  0.0570,  0.0321],
        [-0.0137, -0.0686, -0.0005,  ...,  0.0022,  0.0622, -0.0296],
        [-0.0191, -0.0476,  0.0380,  ...,  0.0519, -0.0476,  0.0076]])
--------------------------------------------------------
tensor([[-0.0097, 

In [65]:
print(main_model.fc2.weight[0:1,0:5])



tensor([[ 0.0630,  0.0016, -0.0621,  0.0142,  0.0037]],
       grad_fn=<SliceBackward0>)


In [66]:
model_dict = dict()
for i in range(number_of_samples):
    model_name="model"+str(i)
    model_info=Net2nn()
    model_dict.update({model_name : model_info })

name_of_models=list(model_dict.keys())


In [72]:

# After the first time run, we should run the following code


# This function downloads all of the local models
# We should first run the client code. Then here.
model_dict = download_local_model_from_each_node(number_of_samples)
print(model_dict["model1"].fc2.weight[0,0:5])
print(model_dict["model0"].fc2.weight[0,0:5])
print("--------------------------------------------------")



main_model = set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_samples) 

print(main_model.fc2.weight[0:1,0:5])
print("--------------------------------------------------")



train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)


test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)



test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy))
print("--------------------------------------------------")


send_main_model_to_nodes_and_update_model_dict(main_model,0) # if 1, then print weight and bias




tensor([ 0.1141, -0.1184, -0.0477,  0.0540, -0.0336], grad_fn=<SliceBackward0>)
tensor([ 0.1188, -0.1231, -0.0392,  0.0608, -0.0465], grad_fn=<SliceBackward0>)
--------------------------------------------------
tensor([[ 0.1152, -0.1199, -0.0452,  0.0578, -0.0391]],
       grad_fn=<SliceBackward0>)
--------------------------------------------------
Iteration 4 : main_model accuracy on all test data:  0.9803
--------------------------------------------------
